# Promethee Outranking Flows
This module shows an example of how to use the "Promethee Outranking" module.

## Definition of inputs and problem formalization

In [25]:
import pandas as pd
from modular_parts.preference import compute_preference_indices
from modular_parts.flows import calculate_promethee_outranking_flows
from core.enums import Direction, GeneralCriterion, FlowType


alternatives = [f"a{i}" for i in range(1, 8)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22]], index=alternatives, columns=criteria)
profiles_performances = pd.DataFrame([[12, 80, 16],
                                      [15, 68, 18],
                                      [18, 55, 20]], index=profiles, columns=criteria)


## Promethee I Flows

### Required preferences
To show all posibilities we compute "alternatives vs alternatives" and "alternatives vs profiles" preferences.

In [26]:
aggregated_alternatives_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)

aggregated_altenatives_vs_profiles_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights,
                                                                    profiles_performances)

In [27]:
aggregated_alternatives_preferences

,a1,a2,a3,a4,a5,a6,a7
a1,0.00,0.88,0.8,0.5,0.0,0.5,0.3
a2,0.00,0.00,0.0,0.0,0.0,0.0,0.0
a3,0.12,0.70,0.0,0.0,0.0,0.5,0.0
a4,0.50,1.00,0.5,0.0,0.0,0.5,0.3
a5,1.00,1.00,1.0,0.5,0.0,0.5,0.8
a6,0.50,0.50,0.5,0.5,0.5,0.0,0.5
a7,0.70,1.00,1.0,0.5,0.0,0.5,0.0


In [28]:
aggregated_altenatives_vs_profiles_preferences[0]

,p1,p2,p3
a1,0.8,0.50,0.5
a2,0.0,0.00,0.0
a3,0.5,0.50,0.0
a4,1.0,0.96,0.0
a5,1.0,0.92,0.5
a6,0.5,0.50,0.5
a7,0.7,0.54,0.5


In [29]:
aggregated_altenatives_vs_profiles_preferences[1]

,a1,a2,a3,a4,a5,a6,a7
p1,0.0,1.0,0.00,0.0,0.0,0.5,0.00
p2,0.2,1.0,0.38,0.0,0.0,0.5,0.30
p3,0.5,1.0,0.50,0.0,0.0,0.5,0.38


### Usage of Promethee I Flows

#### Altenatives vs Alternatives flows
Most commonly used in ranking problems.

In [30]:
alternatives_flows = calculate_promethee_outranking_flows(aggregated_alternatives_preferences,
                                                          flow_type=FlowType.PROMETHEE_I)
alternatives_flows

,positive,negative
a1,0.496667,0.470000
a2,0.000000,0.846667
a3,0.220000,0.633333
a4,0.466667,0.333333
a5,0.800000,0.083333
a6,0.500000,0.416667
a7,0.616667,0.316667


#### Altenatives vs Profiles flows
Most commonly used in sorting problems.

In [31]:
alternatives_vs_profiles_flows = calculate_promethee_outranking_flows(aggregated_altenatives_vs_profiles_preferences,
                                                                      flow_type=FlowType.PROMETHEE_I)
alternatives_vs_profiles_flows

,positive,negative
a1,0.600000,0.233333
a2,0.000000,1.000000
a3,0.333333,0.293333
a4,0.653333,0.000000
a5,0.806667,0.000000
a6,0.500000,0.500000
a7,0.580000,0.226667


## Profile-based outranking flows

### Required preferences

In [32]:
aggregated_altenatives_vs_profiles_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights,
                                                                    profiles_performances)

aggregated_profiles_vs_profiles_preferences, _ =compute_preference_indices(profiles_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)

In [33]:
aggregated_altenatives_vs_profiles_preferences[0]

,p1,p2,p3
a1,0.8,0.50,0.5
a2,0.0,0.00,0.0
a3,0.5,0.50,0.0
a4,1.0,0.96,0.0
a5,1.0,0.92,0.5
a6,0.5,0.50,0.5
a7,0.7,0.54,0.5


In [34]:
aggregated_altenatives_vs_profiles_preferences[1]

,a1,a2,a3,a4,a5,a6,a7
p1,0.0,1.0,0.00,0.0,0.0,0.5,0.00
p2,0.2,1.0,0.38,0.0,0.0,0.5,0.30
p3,0.5,1.0,0.50,0.0,0.0,0.5,0.38


In [35]:
aggregated_profiles_vs_profiles_preferences

,p1,p2,p3
p1,0.0,0.0,0.0
p2,1.0,0.0,0.0
p3,1.0,1.0,0.0


### Usage of Profile-based outranking flows
Those flows are used in sorting and ranking methods, which base on Promethee II method.
In these flows, you can clearly see how the alternative is favored over the profiles.

In [36]:
profile_based_flows = calculate_promethee_outranking_flows(aggregated_altenatives_vs_profiles_preferences,
                                                           flow_type=FlowType.PROMETHEE_II,
                                                           profiles_preferences=aggregated_profiles_vs_profiles_preferences)
profile_based_flows

positive  negative
Ra1 p1  0.000000  0.933333
    p2  0.400000  0.500000
    p3  0.833333  0.166667
    a1  0.600000  0.233333
Ra2 p1  0.333333  0.666667
    p2  0.666667  0.333333
    p3  1.000000  0.000000
    a2  0.000000  1.000000
Ra3 p1  0.000000  0.833333
    p2  0.460000  0.500000
    p3  0.833333  0.000000
    a3  0.333333  0.293333
Ra4 p1  0.000000  1.000000
    p2  0.333333  0.653333
    p3  0.666667  0.000000
    a4  0.653333  0.000000
Ra5 p1  0.000000  1.000000
    p2  0.333333  0.640000
    p3  0.666667  0.166667
    a5  0.806667  0.000000
Ra6 p1  0.166667  0.833333
    p2  0.500000  0.500000
    p3  0.833333  0.166667
    a6  0.500000  0.500000
Ra7 p1  0.000000  0.900000
    p2  0.433333  0.513333
    p3  0.793333  0.166667
    a7  0.580000  0.226667